Adapted from https://www.learnpytorch.io/06_pytorch_transfer_learning/ and https://www.learnpytorch.io/05_pytorch_going_modular/#2-create-datasets-and-dataloaders-data_setuppy

In [1]:
!pip install torchinfo


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import shutil
import random
import math
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets, transforms
from torch import nn
from torchvision import transforms
import os
import zipfile
from pathlib import Path
import requests
from torch.utils.data import DataLoader
from torchinfo import summary
from timeit import default_timer as timer
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple 

In [ ]:
def split_data(source_folder, output_folder, train_ratio=0.8):
    """
    Splits image data from subfolders in a source folder into train and test sets
    in an output folder, maintaining the subfolder structure.

    Args:
        source_folder (str): Path to the folder containing the class subfolders.
        output_folder (str): Path to the folder where 'train' and 'test' folders
                             will be created.
        train_ratio (float): Proportion of images to allocate to the training set
                             (default is 0.8 for 80%).
    """
    print(f"Source folder: {os.path.abspath(source_folder)}")
    print(f"Output folder: {os.path.abspath(output_folder)}")
    print(f"Train ratio: {train_ratio}")

    if not os.path.exists(source_folder):
        print(f"Error: Source folder '{source_folder}' does not exist.")
        return

    train_dir = os.path.join(output_folder, 'train')
    test_dir = os.path.join(output_folder, 'test')

    try:
        os.makedirs(output_folder, exist_ok=True)
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)
        print(f"Created base directories: '{train_dir}' and '{test_dir}'")
    except OSError as e:
        print(f"Error creating directories: {e}")
        return

    for class_name in os.listdir(source_folder):
        source_class_path = os.path.join(source_folder, class_name)

        if os.path.isdir(source_class_path):
            print(f"\nProcessing class: {class_name}")
            train_class_path = os.path.join(train_dir, class_name)
            test_class_path = os.path.join(test_dir, class_name)
            os.makedirs(train_class_path, exist_ok=True)
            os.makedirs(test_class_path, exist_ok=True)
            try:
                all_files = [
                    f for f in os.listdir(source_class_path)
                    if os.path.isfile(os.path.join(source_class_path, f))
                ]
            except OSError as e:
                print(f"Warning: Could not read files in {source_class_path}. Skipping. Error: {e}")
                continue


            if not all_files:
                print(f"Warning: No files found in {source_class_path}. Skipping.")
                continue

            random.shuffle(all_files)

            split_index = math.floor(len(all_files) * train_ratio)
            train_files = all_files[:split_index]
            test_files = all_files[split_index:]

            print(f"Total files: {len(all_files)}")
            print(f"Train files: {len(train_files)}")
            print(f"Test files : {len(test_files)}")

            print(f"Copying {len(train_files)} files to {train_class_path}...")
            copied_train = 0
            for file_name in train_files:
                source_file_path = os.path.join(source_class_path, file_name)
                dest_file_path = os.path.join(train_class_path, file_name)
                try:
                    shutil.copy2(source_file_path, dest_file_path)
                    copied_train += 1
                except Exception as e:
                    print(f"Error copying {source_file_path} to {dest_file_path}: {e}")
            if copied_train != len(train_files):
                 print(f"Warning: Only {copied_train} out of {len(train_files)} train files were copied successfully.")

            print(f"Copying {len(test_files)} files to {test_class_path}...")
            copied_test = 0
            for file_name in test_files:
                source_file_path = os.path.join(source_class_path, file_name)
                dest_file_path = os.path.join(test_class_path, file_name)
                try:
                    shutil.copy2(source_file_path, dest_file_path)
                    copied_test += 1
                except Exception as e:
                    print(f"Error copying {source_file_path} to {dest_file_path}: {e}")
            if copied_test != len(test_files):
                 print(f"Warning: Only {copied_test} out of {len(test_files)} test files were copied successfully.")


        else:
            print(f"Skipping '{class_name}' as it is not a directory.")

    print("\nData splitting complete!")


In [ ]:
source_directory = "D:\\Study\\resics_applied_ml_project\\pytorch_data\\NWPU-RESISC45"
output_directory = "D:\\Study\\resics_applied_ml_project\\splitted_data"
training_ratio = 0.8
if 'path/to/your' in source_directory or 'path/to/your' in output_directory:
    print("="*50)
    print("ERROR: Please update the 'source_directory' and 'output_directory'")
    print("variables in the script with your actual folder paths.")
    print("="*50)
else:
    split_data(source_directory, output_directory, training_ratio)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
image_path = Path("D:\\Study\\resics_applied_ml_project\\splitted_data")
train_dir = image_path / "train/"
test_dir = image_path / "test/"

In [ ]:
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
NUM_WORKERS = os.cpu_count()
def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  class_names = train_data.classes

  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

In [ ]:
train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms, 
                                                                               batch_size=32)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x2052717d2d0>,
 ['airplane',
  'airport',
  'baseball_diamond',
  'basketball_court',
  'beach',
  'bridge',
  'chaparral',
  'church',
  'circular_farmland',
  'cloud',
  'commercial_area',
  'dense_residential',
  'desert',
  'forest',
  'freeway',
  'golf_course',
  'ground_track_field',
  'harbor',
  'industrial_area',
  'intersection',
  'island',
  'lake',
  'meadow',
  'medium_residential',
  'mobile_home_park',
  'mountain',
  'overpass',
  'palace',
  'parking_lot',
  'railway',
  'railway_station',
  'rectangular_farmland',
  'river',
  'roundabout',
  'runway',
  'sea_ice',
  'ship',
  'snowberg',
  'sparse_residential',
  'stadium',
  'storage_tank',
  'tennis_court',
  'terrace',
  'thermal_power_station',
  'wetland'])

In [10]:
weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2

In [11]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
a, b, c = create_dataloaders(train_dir=train_dir,
                                        test_dir=test_dir,
                                        transform=auto_transforms,
                                        batch_size=32)

assert type(a) is type(train_dataloader) 
assert type(b) is type(test_dataloader) 
assert type(c) is type(class_names)

In [13]:
model = torchvision.models.resnet50(weights=weights).to(device)

In [14]:
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 45)
model = model.to(device)
for param in model.layer4.parameters():
    param.requires_grad = True

In [ ]:
summary(model=model, 
        input_size=(32, 3, 224, 224),
        # col_names=["input_size"],
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 45]             --                   Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    └─Bottleneck (0)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     (4,096)              False
│    │    

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  model.train()

  train_loss, train_acc = 0, 0
  for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  model.eval() 

  test_loss, test_acc = 0, 0

  with torch.inference_mode():
      for batch, (X, y) in enumerate(dataloader):
          X, y = X.to(device), y.to(device)

          test_pred_logits = model(X)

          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]} 
    For example if training for epochs=2: 
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]} 
  """
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  return results

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

start_time = timer()

results = train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=20,
                       device=device)

end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.3132 | train_acc: 0.7129 | test_loss: 0.7284 | test_acc: 0.8237
Epoch: 2 | train_loss: 0.6282 | train_acc: 0.8376 | test_loss: 0.5646 | test_acc: 0.8516
Epoch: 3 | train_loss: 0.4893 | train_acc: 0.8667 | test_loss: 0.4924 | test_acc: 0.8646
Epoch: 4 | train_loss: 0.4090 | train_acc: 0.8892 | test_loss: 0.4526 | test_acc: 0.8751
Epoch: 5 | train_loss: 0.3535 | train_acc: 0.9028 | test_loss: 0.4216 | test_acc: 0.8746
Epoch: 6 | train_loss: 0.3114 | train_acc: 0.9140 | test_loss: 0.4481 | test_acc: 0.8736
Epoch: 7 | train_loss: 0.2788 | train_acc: 0.9243 | test_loss: 0.4035 | test_acc: 0.8832
Epoch: 8 | train_loss: 0.2556 | train_acc: 0.9314 | test_loss: 0.4306 | test_acc: 0.8770
Epoch: 9 | train_loss: 0.2343 | train_acc: 0.9369 | test_loss: 0.4068 | test_acc: 0.8808
Epoch: 10 | train_loss: 0.2197 | train_acc: 0.9392 | test_loss: 0.4000 | test_acc: 0.8827
Epoch: 11 | train_loss: 0.2012 | train_acc: 0.9444 | test_loss: 0.3760 | test_acc: 0.8862
Epoch: 12 | train_l

In [ ]:
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

model_save_path = os.path.join(output_dir, "trained_resnet50_model.pth")
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

results_save_path = os.path.join(output_dir, "training_results.pth")
torch.save(results, results_save_path)
print(f"Training results saved to {results_save_path}")

Model saved to ./output\trained_resnet50_model.pth
Training results saved to ./output\training_results.pth


The following part with model loading should be tested in the future.

In [ ]:
# import torch
# import torchvision

# model_load_path = "./output/trained_resnet50_model.pth"  # Ensure this matches where you saved it

# weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2
# model = torchvision.models.resnet50(weights=weights).to(device)
# for param in model.parameters():
#     param.requires_grad = False
# num_ftrs = model.fc.in_features
# model.fc = torch.nn.Linear(num_ftrs, 45)
# model = model.to(device)
# for param in model.layer4.parameters():
#     param.requires_grad = True

# loaded_model_state_dict = torch.load(model_load_path)

# model.load_state_dict(loaded_model_state_dict)

# model.eval()

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)


# results_load_path = "./output/training_results.pth"
# loaded_results = torch.load(results_load_path)
# print("Training results loaded")